In [2]:
import requests

In [3]:
url_1 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2020.zip"
url_2 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2019.zip"
url_3 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2018.zip"
url_4 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2017.zip"
url_5 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2016.zip"

In [4]:
dado_1 = requests.get(url_1, verify=False)
dado_2 = requests.get(url_2, verify=False)
dado_3 = requests.get(url_3, verify=False)
dado_4 = requests.get(url_4, verify=False)
dado_5 = requests.get(url_5, verify=False)

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/urllib3/conn

In [5]:
import zipfile

with open('dado_1.zip', 'wb') as f:
	f.write(dado_1.content)
	
with open('dado_2.zip', 'wb') as f:
	f.write(dado_2.content)

with open('dado_3.zip', 'wb') as f:
	f.write(dado_3.content)

with open('dado_4.zip', 'wb') as f:
	f.write(dado_4.content)

with open('dado_5.zip', 'wb') as f:
	f.write(dado_5.content)
	
with zipfile.ZipFile('dado_1.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")
	
with zipfile.ZipFile('dado_2.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")

with zipfile.ZipFile('dado_3.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")

with zipfile.ZipFile('dado_4.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")

with zipfile.ZipFile('dado_5.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lower

In [7]:
spark = SparkSession \
        .builder \
        .config("spark.sql.repl.eagerEval.enabled", "True") \
        .config("spark.sql.repl.eagerEval.maxNumRows", "150") \
        .getOrCreate()


In [8]:
df_1 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2020/dados/MICRODADOS_CADASTRO_IES_2020.CSV", sep=';', header=True, inferSchema=True)

df_2 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2019/dados/MICRODADOS_CADASTRO_IES_2019.CSV", sep=';', header=True, inferSchema=True)

df_3 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2018/dados/MICRODADOS_CADASTRO_IES_2018.CSV", sep=';', header=True, inferSchema=True)

df_4 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2017/dados/MICRODADOS_CADASTRO_IES_2017.CSV", sep=';', header=True, inferSchema=True)

df_5 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2016/dados/MICRODADOS_CADASTRO_IES_2016.CSV", sep=';', header=True, inferSchema=True)


In [9]:
for col in df_1.columns:
    df_1 = df_1.withColumnRenamed(col, col.lower())
for col in df_2.columns:
    df_2 = df_2.withColumnRenamed(col, col.lower())
for col in df_3.columns:
    df_3 = df_3.withColumnRenamed(col, col.lower())
for col in df_4.columns:
    df_4 = df_4.withColumnRenamed(col, col.lower())
for col in df_5.columns:
    df_5 = df_5.withColumnRenamed(col, col.lower())


In [10]:
df_parcial_1 = df_1.union(df_2)

In [11]:
df_parcial_2 = df_parcial_1.union(df_3)

In [12]:
df_parcial_3 = df_parcial_2.union(df_4)

In [13]:
df_final = df_parcial_3.union(df_5)

In [14]:
df_final.write.parquet("MICRODADOS_IES_CONSULTA2.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/jovyan/work/MICRODADOS_IES_CONSULTA2.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [15]:
df_final1 = spark.read.parquet("MICRODADOS_IES_CONSULTA2.parquet")

In [16]:
df_final1.printSchema()

root
 |-- nu_ano_censo: integer (nullable = true)
 |-- no_regiao_ies: string (nullable = true)
 |-- co_regiao_ies: integer (nullable = true)
 |-- no_uf_ies: string (nullable = true)
 |-- sg_uf_ies: string (nullable = true)
 |-- co_uf_ies: integer (nullable = true)
 |-- no_municipio_ies: string (nullable = true)
 |-- co_municipio_ies: integer (nullable = true)
 |-- in_capital_ies: integer (nullable = true)
 |-- no_mesorregiao_ies: string (nullable = true)
 |-- co_mesorregiao_ies: integer (nullable = true)
 |-- no_microrregiao_ies: string (nullable = true)
 |-- co_microrregiao_ies: integer (nullable = true)
 |-- tp_organizacao_academica: integer (nullable = true)
 |-- tp_categoria_administrativa: integer (nullable = true)
 |-- no_mantenedora: string (nullable = true)
 |-- co_mantenedora: integer (nullable = true)
 |-- co_ies: integer (nullable = true)
 |-- no_ies: string (nullable = true)
 |-- sg_ies: string (nullable = true)
 |-- ds_endereco_ies: string (nullable = true)
 |-- ds_numero_

In [17]:
df_final1.createOrReplaceTempView("parquet_df_final")

### CONSULTAS REFERENTE A PESQUISA:
# Os impactos da pandemia no Ensino Superior do Brasil
## Análise comparativa do Ensino Superior no Brasil nos anos de 2019 e 2020
#

### 1. Houve crescimento ou queda no número de instituições? Qual estado teve o maior crescimento?

In [18]:
spark.sql("SELECT t.Estado, t.Num_instituicoes_2016, t.Num_instituicoes_2017, t.Num_instituicoes_2018, t.Num_instituicoes_2019, t.Num_instituicoes_2020, t.Num_instituicoes_2020 - t.Num_instituicoes_2019 AS Diferenca_2020_2019 \
           FROM ( \
               SELECT sg_uf_ies AS Estado, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN 1 ELSE 0 END) AS Num_instituicoes_2020, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN 1 ELSE 0 END) AS Num_instituicoes_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2018 THEN 1 ELSE 0 END) AS Num_instituicoes_2018, \
                   SUM(CASE WHEN nu_ano_censo = 2017 THEN 1 ELSE 0 END) AS Num_instituicoes_2017, \
                   SUM(CASE WHEN nu_ano_censo = 2016 THEN 1 ELSE 0 END) AS Num_instituicoes_2016 \
               FROM parquet_df_final \
               GROUP BY sg_uf_ies \
           ) t \
           order by Diferenca_2020_2019 desc")

Estado,Num_instituicoes_2016,Num_instituicoes_2017,Num_instituicoes_2018,Num_instituicoes_2019,Num_instituicoes_2020,Diferenca_2020_2019
SC,94,93,97,100,104,4
AM,19,20,23,23,25,2
AP,16,15,15,14,15,1
RR,7,7,8,8,8,0
MS,34,28,33,35,34,-1
AL,28,29,29,31,30,-1
RN,28,28,29,29,28,-1
TO,24,24,24,26,24,-2
PA,47,54,57,73,71,-2
SE,18,18,19,20,18,-2


### 2. Quantas bibliotecas passaram a oferecer serviços pela internet entre os anos de 2016 e 2020?

In [19]:
spark.sql("select  nu_ano_censo as Ano_pesquisa, count(in_servico_internet) as Qtd_bibliot_ofertam_servicos_pela_internet \
from parquet_df_final \
where in_servico_internet = 1 and (nu_ano_censo = 2019 or nu_ano_censo = 2020 or nu_ano_censo = 2018 or nu_ano_censo = 2017 or nu_ano_censo = 2016) \
group by nu_ano_censo order by nu_ano_censo")

Ano_pesquisa,Qtd_bibliot_ofertam_servicos_pela_internet
2016,2189
2017,2262
2018,2353
2019,2460
2020,2339


### 3. Houve quantos contratos ativos entre os anos de 2016 e 2020? O número de contratos de 2020 superou 2019?

In [20]:
spark.sql("select sum(qt_doc_total) as Contrato_docentes, nu_ano_censo as Ano_pesquisa \
            from parquet_df_final \
            group by nu_ano_censo \
            order by nu_ano_censo")

Contrato_docentes,Ano_pesquisa
397611,2016
392036,2017
397893,2018
399428,2019
378492,2020


### 4. Qual estado teve o maior número de admissões de docentes entre 35 e 54 anos, em 2020.

In [21]:
spark.sql("SELECT t.sg_uf_ies as Estado, t.Docentes_35_54_2019, t.Docentes_35_54_2020, t.Docentes_35_54_2020 - t.Docentes_35_54_2019 AS Diferenca \
           FROM ( \
               SELECT sg_uf_ies, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN qt_doc_ex_35_39 + qt_doc_ex_40_44 + qt_doc_ex_45_49 + qt_doc_ex_50_54 ELSE 0 END) AS Docentes_35_54_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN qt_doc_ex_35_39 + qt_doc_ex_40_44 + qt_doc_ex_45_49 + qt_doc_ex_50_54 ELSE 0 END) AS Docentes_35_54_2020 \
               FROM parquet_df_final \
               WHERE nu_ano_censo = 2019 OR nu_ano_censo = 2020 \
               GROUP BY sg_uf_ies \
           ) t \
           ORDER BY diferenca DESC \
           LIMIT 54")

Estado,Docentes_35_54_2019,Docentes_35_54_2020,Diferenca
SC,11136,11274,138
PB,5822,5936,114
SE,2451,2488,37
AM,3117,3135,18
RN,3889,3903,14
CE,7967,7970,3
PE,8316,8308,-8
AP,1031,1017,-14
RR,782,754,-28
MS,3649,3601,-48


### 5. Quais tipos de organização acadêmica foram mais prejudicadas com demissões de funcionários em 2020? Informe o tipo de categoria administrativa.

#### NUM DO TIPO DE ORG. ACADÊMICA
##### 1 - Universidade
##### 2 - Centro Universitário
##### 3 - Faculdade
##### 4 - Instituto Federal de Educação, Ciência e Tecnologia
##### 5 - Centro Federal de Educação Tecnológica

#### NUM DO TIPO DE CATEGORIA ADMINISTRATIVA
##### 1 - Pública Federal
##### 2 - Pública Estadual
##### 3 - Pública Municipal
##### 4 - Privada com fins lucrativos
##### 5 - Privada sem fins lucrativos
##### 6 - Privada - Particular sem sentido estrito
##### 7 - Especial
##### 8 - Privada comunitária
##### 9 - Privada confessional

In [22]:
spark.sql("SELECT t.tp_categoria_administrativa as Tipo_categoria_adm, t.tp_organizacao_academica as Tipo_org_academica, t.Contrato_docentes_2019, t.Contrato_docentes_2020, t.Contrato_docentes_2020 - t.Contrato_docentes_2019 AS Diferenca \
           FROM ( \
               SELECT tp_organizacao_academica, tp_categoria_administrativa, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN qt_doc_total ELSE 0 END) AS Contrato_docentes_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN qt_doc_total ELSE 0 END) AS Contrato_docentes_2020 \
               FROM parquet_df_final \
               WHERE nu_ano_censo = 2019 OR nu_ano_censo = 2020 \
               GROUP BY tp_organizacao_academica, tp_categoria_administrativa  \
               ORDER BY tp_organizacao_academica, tp_categoria_administrativa \
           ) t")

Tipo_categoria_adm,Tipo_org_academica,Contrato_docentes_2019,Contrato_docentes_2020,Diferenca
1,1,99244,96374,-2870
2,1,46528,44516,-2012
3,1,2637,2457,-180
4,1,20998,20443,-555
5,1,43760,41436,-2324
1,2,140,124,-16
2,2,105,133,28
3,2,711,486,-225
4,2,31125,32534,1409
5,2,26446,25782,-664
